<a href="https://colab.research.google.com/github/Aksinhaa/Pony/blob/main/NGS_colab_part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Introduction to Variant Filtering

High-throughput sequencing technologies generate millions of genetic variants across genomes, but not all detected variants are biologically meaningful or technically reliable. Raw variant calls often contain sequencing errors, low-confidence genotypes, and missing data that can severely bias downstream analyses. From removing errors to retain only high-confidence SNPs, each step here helps you build a more accurate dataset for further applications such as population genetics.
Below are the details of the filters that we are going to use during this analysis to get a filtered VCF file.

1: *passOnly*-  Retained only variant sites that passed all internal quality filter performed by the variant caller (Strelka in this case). During variant calling, Strelka assigns a PASS or FAIL tag to each variant based on multiple internal metrics such as base quality, mapping quality, strand bias, and read position bias.

2: *biallelicOnly*- Kept only biallelic variants (i.e., sites with exactly two alleles) for downstream compatibility and clarity. Because most population genetic software assumes biallelic markers, this filter ensures mathematical simplicity, software compatibility, and interpretability.

3: *rmvIndels*- Removed insertions and deletions (INDELs), keeping only SNPs (single nucleotide polymorphisms). Indels have higher sequencing and alignment error rates than SNPs. Removing indels ensures that the dataset is composed of high-confidence point mutations.

4: *minMAF0Pt05*- Retained only variants with minor allele frequency (MAF) ≥ 0.05, meaning the alternate allele must be present in at least 5% of individuals. As very rare variants (MAF < 5%) are more likely to be sequencing errors, they are often poorly powered in statistical tests.

5: *chr_E2*- Restricted to a specific chromosome/region of interest (e.g., chr_E2). This enables targeted analysis of specific genomic regions.

6: *minDP3*- Required a minimum depth (DP) of 3 per genotype to avoid false positives due to low coverage. Because low-depth genotypes are highly error-prone and vulnerable to random sequencing noise.

7: *minQ30*- Ensured that each site has a minimum site quality score of 30, reflecting high confidence in the variant.  This improves accuracy of allele frequency estimates and reliability of genome-wide scans.

8: *minGQ30*- Filtered genotypes to retain only those with a Genotype Quality (GQ) ≥ 30, removing uncertain genotype calls. This helps prevent the misclassification of homozygous vs heterozygous states.

9: *hwe_0.05*- Removed variants deviating significantly from Hardy-Weinberg Equilibrium (p < 0.05), which could indicate genotyping errors or population substructure.

10: *imiss_0.6*- Filtered out individuals with >60% missing genotypes to maintain data quality. Highly incomplete samples can distort PCA and admixture results.

11: *miss_0.6*- Removed variants missing in >60% of individuals. This ensures each retained SNP is well represented across individuals.

12: *mid95percentile* - Likely indicates the removal of extreme outliers in genotype depth or quality, retaining the middle 95% of the data. This removes the Lowest 2.5% of undercovered, unreliable sites and the highest 2.5% of multi-mapped, repetitive, or duplicated regions.





###Prerequisites and setup:
To run this variant-filtering command line workflow, make sure you have Miniconda already installed, and it is working. Must have already downloaded the required input VCF file from Zenodo into your working directory.
The setup involves creating and activating the required conda environments for vcftools, ggplot2, and base R, installing these packages via the bioconda and Cond forge channels.

First, we'll download and install Miniconda. We use `wget` to download the installer script and then execute it using `bash`.

In [44]:
# Miniconda installation and environment setup for Colab NGS Workshop

# Download and install Miniconda (skip if already installed)
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
!bash miniconda.sh -b -p /usr/local/miniconda

import sys, os
sys.path.append('/usr/local/miniconda/lib/python3.8/site-packages')
os.environ['PATH'] = "/usr/local/miniconda/bin:" + os.environ['PATH']

# Accept ToS for main and R conda channels
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

# Install necessary bioinformatics tools into the environment
!conda create -n vcf_filter -c bioconda vcftools


ERROR: File or directory already exists: '/usr/local/miniconda'
If you want to update an existing installation, use the -u option.
accepted Terms of Service for https://repo.anaconda.com/pkgs/main
accepted Terms of Service for https://repo.anaconda.com/pkgs/r
Jupyter detected...
2 channel Terms of Service accepted

Remove existing environment?
This will remove ALL directories contained within this specified prefix directory, including any other conda environments.

 (y/[n])? y

Channels:
 - bioconda
 - defaults
Platform: linux-64
Solving environment: / done


==> WARNING: A newer version of conda exists. <==
    current version: 25.9.1
    latest version: 25.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local/miniconda/envs/vcf_filter

  added / updated specs:
    - vcftools


The following NEW packages will be INSTALLED:

  _libgcc_mutex      pkgs/main/linux-64::_libgcc_mutex-0.1-main 
  _openmp

First, let's list all conda environments to confirm `vcf_filter` is present.

In [45]:
!conda env list


# conda environments:
#
# *  -> active
# + -> frozen
base                     /usr/local/miniconda
R_env                    /usr/local/miniconda/envs/R_env
ggplot2                  /usr/local/miniconda/envs/ggplot2
vcf_filter               /usr/local/miniconda/envs/vcf_filter



Next, we will list the packages installed in the `vcf_filter` environment to ensure `vcftools` is there.

In [46]:
!conda list -n vcf_filter

# packages in environment at /usr/local/miniconda/envs/vcf_filter:
#
# Name                     Version          Build             Channel
_libgcc_mutex              0.1              main
_openmp_mutex              5.1              1_gnu
libgcc                     15.2.0           h69a1729_7
libgcc-ng                  15.2.0           h166f726_7
libgomp                    15.2.0           h4751f2c_7
libnsl                     2.0.0            h5eee18b_0
libstdcxx                  15.2.0           h39759b7_7
libzlib                    1.3.1            hb25bd0a_0
perl                       5.32.1           0_h5eee18b_perl5
vcftools                   0.1.17           pl5321h077b44d_0  bioconda


In [47]:
# Create the directory if it doesn't exist
!mkdir -p vcf_file

# Download the VCF file into the created directory
!wget -P vcf_file https://zenodo.org/records/15173226/files/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_noIndels_minMAF0Pt05_chr_E2_minDP3.recode.vcf.gz

--2025-12-11 10:18:38--  https://zenodo.org/records/15173226/files/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_noIndels_minMAF0Pt05_chr_E2_minDP3.recode.vcf.gz
Resolving zenodo.org (zenodo.org)... 188.185.43.153, 137.138.52.235, 188.185.48.75, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104695078 (100M) [application/octet-stream]
Saving to: ‘vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_noIndels_minMAF0Pt05_chr_E2_minDP3.recode.vcf.gz.1’

machali_Aligned_ran 100%[===================>]  99.84M  1.08MB/s    in 1m 43s  

2025-12-11 10:20:21 (997 KB/s) - ‘vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_noIndels_minMAF0Pt05_chr_E2_minDP3.recode.vcf.gz.1’ saved [104695078/104695078]



First, we'll apply base quality (`--minQ`), genotype quality (`--minGQ`), and Hardy-Weinberg equilibrium (`--hwe`) filters to the VCF file. The output file name will reflect these new filters.

In [48]:
%%bash
# Activate conda environment
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh
conda activate vcf_filter

# Define input and output files
input_vcf_gz="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_noIndels_minMAF0Pt05_chr_E2_minDP3.recode.vcf.gz"
output_prefix_step1="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_noIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05"

# Run vcftools filters
vcftools --gzvcf "$input_vcf_gz" \
         --minQ 30 \
         --minGQ 30 \
         --hwe 0.05 \
         --recode \
         --out "$output_prefix_step1"


VCFtools - 0.1.17
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_noIndels_minMAF0Pt05_chr_E2_minDP3.recode.vcf.gz
	--max-alleles 2
	--minGQ 30
	--hwe 0.05
	--minQ 30
	--out vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_noIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05
	--recode

Using zlib version: 1.3.1
After filtering, kept 60 out of 60 Individuals
Outputting VCF file...
After filtering, kept 138293 out of a possible 186729 Sites
Run Time = 64.00 seconds


Next, we will remove indels (insertions and deletions) from the filtered VCF file. The `--remove-indels` flag ensures that only SNP (Single Nucleotide Polymorphism) sites are retained. The output file name will again be updated to reflect this filter.

In [49]:
%%bash
# Activate conda environment
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh
conda activate vcf_filter

# Define input and output files
input_vcf_step1="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_noIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05.recode.vcf"
output_prefix_step2="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05"

# Remove INDELs
vcftools --vcf "$input_vcf_step1" \
         --remove-indels \
         --recode \
         --out "$output_prefix_step2"


VCFtools - 0.1.17
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_noIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05.recode.vcf
	--out vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05
	--recode
	--remove-indels

After filtering, kept 60 out of 60 Individuals
Outputting VCF file...
After filtering, kept 112530 out of a possible 138293 Sites
Run Time = 49.00 seconds


Finally, we will apply the individual missingness filter. This command will generate an output file with a `.imiss` extension, which contains information about the fraction of missing sites for each individual.

In [50]:
%%bash
# Activate conda environment
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh
conda activate vcf_filter

# Define input and output
input_vcf_step2="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05.recode.vcf"
output_prefix_step2="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05"

# Generate .imiss file
vcftools --vcf "$input_vcf_step2" \
         --missing-indv \
         --out "$output_prefix_step2"


VCFtools - 0.1.17
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05.recode.vcf
	--missing-indv
	--out vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05

After filtering, kept 60 out of 60 Individuals
Outputting Individual Missingness
After filtering, kept 112530 out of a possible 112530 Sites
Run Time = 1.00 seconds


The command above will produce a file named `machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05.imiss`. You can inspect this file to see the fraction of missing sites for each individual in the `F_MISS` column. This information is crucial for identifying individuals with a high proportion of missing data, which might be excluded from further analysis if the missingness exceeds a certain threshold.

Now, we will remove individuals that have a missing data proportion greater than 60%. This involves using `awk` to parse the `.imiss` file, identify individuals with `F_MISS` (fraction of missing data, which is the 5th column) greater than 0.6, and then passing these individual IDs to `vcftools` using the `--remove` flag.

In [51]:
%%bash
# Activate conda environment
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh
conda activate vcf_filter

# File paths
input_vcf_step2="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05.recode.vcf"
imiss_file="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05.imiss"
output_prefix_step3="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6"
temp_remove_list="vcf_file/individuals_to_remove.txt"

# Extract individuals with >60% missing data
awk 'NR>1 && $5>0.6 {print $1}' "$imiss_file" > "$temp_remove_list"

# Remove those individuals from the VCF
vcftools --vcf "$input_vcf_step2" \
         --remove "$temp_remove_list" \
         --recode \
         --out "$output_prefix_step3"

# Cleanup temporary file
rm "$temp_remove_list"


VCFtools - 0.1.17
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05.recode.vcf
	--remove vcf_file/individuals_to_remove.txt
	--out vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6
	--recode

Excluding individuals in 'exclude' list
After filtering, kept 53 out of 60 Individuals
Outputting VCF file...
After filtering, kept 112530 out of a possible 112530 Sites
Run Time = 39.00 seconds


After this step, a new VCF file will be generated (prefixed with `machali_Aligned...imiss_0.6`) that excludes individuals with more than 60% missing data. This helps to improve the quality of downstream analyses by ensuring that only individuals with a sufficient amount of genotyped data are included.

In [52]:
%%bash
# Activate conda environment
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh
conda activate vcf_filter

# Input VCF (after removing individuals >60% missing)
input_vcf="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6.recode.vcf"

# Loop over missingness thresholds (10% to 90%)
for miss in {10..90..10}; do

    # Convert percent to decimal
    perc=$(echo "scale=2; $miss / 100" | bc)

    # Output prefix for this missingness threshold
    output_prefix="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6_miss_${miss}"

    # Run VCFtools missingness filter
    vcftools --vcf "$input_vcf" \
             --max-missing "$perc" \
             --recode \
             --out "$output_prefix"
    echo "Completed missingness filter: $miss% (max-missing $perc)"
done

Completed missingness filter: 10% (max-missing .10)
Completed missingness filter: 20% (max-missing .20)
Completed missingness filter: 30% (max-missing .30)
Completed missingness filter: 40% (max-missing .40)
Completed missingness filter: 50% (max-missing .50)
Completed missingness filter: 60% (max-missing .60)
Completed missingness filter: 70% (max-missing .70)
Completed missingness filter: 80% (max-missing .80)
Completed missingness filter: 90% (max-missing .90)



VCFtools - 0.1.17
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6.recode.vcf
	--max-missing 0.1
	--out vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6_miss_10
	--recode

After filtering, kept 53 out of 53 Individuals
Outputting VCF file...
After filtering, kept 110654 out of a possible 112530 Sites
Run Time = 40.00 seconds

VCFtools - 0.1.17
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6.recode.vcf
	--max-missing 0.2
	--out vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_

In [54]:
%%bash
# Remove old counts file if it exists
rm -f variant_counts.txt

# Loop over missingness thresholds
for miss in {10..90..10}; do

    # Input VCF for this threshold
    vcf_file="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6_miss_${miss}.recode.vcf"

    # Count variants (non-header lines)
    count=$(grep -vc "^#" "$vcf_file")

    # Save: missingness  variant_count
    echo "${miss} ${count}" >> vcf_file/variant_counts.txt

    echo "Counted variants for ${miss}% missingness → ${count}"

done

Counted variants for 10% missingness → 110654
Counted variants for 20% missingness → 109252
Counted variants for 30% missingness → 108069
Counted variants for 40% missingness → 106013
Counted variants for 50% missingness → 103569
Counted variants for 60% missingness → 99691
Counted variants for 70% missingness → 88717
Counted variants for 80% missingness → 64660
Counted variants for 90% missingness → 21455


In [ ]:
%%bash
# Add conda to PATH
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh

# Create the ggplot2 environment
conda create -y -n ggplot2 -c conda-forge r-ggplot2

# Activate the environment
conda activate ggplot2

Jupyter detected...
2 channel Terms of Service accepted
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: \ | done

## Package Plan ##

  environment location: /usr/local/miniconda/envs/ggplot2

  added / updated specs:
    - r-ggplot2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |            2_gnu          23 KB  conda-forge
    _r-mutex-1.0.1             |      anacondar_1           3 KB  conda-forge
    binutils_impl_linux-64-2.45|default_hfdba357_104         3.6 MB  conda-forge
    bwidget-1.10.1             |       ha770c72_1         127 KB  conda-forge
    bzip2-1.0.8                |       hda65f42_8         254 KB  conda-forge
    c-ares-1.34.6              |       hb03c661_0         203 KB  conda-forge
    ca-certificates-2025.11.12 |      



==> WARNING: A newer version of conda exists. <==
    current version: 25.9.1
    latest version: 25.11.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [56]:
%%bash
# Activate conda ggplot2 environment
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh
conda activate ggplot2

# Create an R script to run ggplot2
cat << 'EOF' > plot_variants.R
library(ggplot2)
library(scales)

data <- read.table("vcf_file/variant_counts.txt", header=FALSE,
                   col.names=c("Missingness", "Variants"))

p <- ggplot(data, aes(x=Missingness, y=Variants)) +
     geom_line(color="blue") +
     geom_point(color="red") +
     ggtitle("Number of Passed Variants vs. Missingness Filter") +
     xlab("Max Missingness (%)") +
     ylab("Number of Variants") +
     scale_y_continuous(labels = comma) +
     scale_x_continuous(limits = c(10, 90)) +
     theme_minimal()

ggsave("vcf_file/variant_plot.png", plot = p)
EOF

# Run the R script
Rscript plot_variants.R

Saving 7 x 7 in image


In [57]:
%%bash
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh
conda activate vcf_filter

# Use the correct miss_60 file
input_vcf="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6_miss_60.recode.vcf"

output_prefix="vcf_file/depth"

vcftools --vcf "$input_vcf" \
         --site-mean-depth \
         --out "$output_prefix"



VCFtools - 0.1.17
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6_miss_60.recode.vcf
	--out vcf_file/depth
	--site-mean-depth

After filtering, kept 53 out of 53 Individuals
Outputting Depth for Each Site
After filtering, kept 99691 out of a possible 99691 Sites
Run Time = 2.00 seconds


In [58]:
%%bash
# Add conda to PATH and initialize shell
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh

# Create R environment (run once)
conda create -y -n R_env -c conda-forge r-base

# Activate R environment
conda activate R_env

# Create an R script to calculate middle 95% depth
cat << 'EOF' > compute_depth_range.R
depth_data <- read.table("vcf_file/depth.ldepth.mean", header = TRUE)

depths <- depth_data$MEAN_DEPTH

lower_bound <- quantile(depths, 0.025, na.rm = TRUE)
upper_bound <- quantile(depths, 0.975, na.rm = TRUE)

cat("Middle 95% range:", lower_bound, "to", upper_bound, "\n")
EOF

# Run the R script
Rscript compute_depth_range.R


Jupyter detected...
2 channel Terms of Service accepted
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: - done

## Package Plan ##

  environment location: /usr/local/miniconda/envs/R_env

  added / updated specs:
    - r-base


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  _r-mutex           conda-forge/noarch::_r-mutex-1.0.1-anacondar_1 
  binutils_impl_lin~ conda-forge/linux-64::binutils_impl_linux-64-2.45-default_hfdba357_104 
  bwidget            conda-forge/linux-64::bwidget-1.10.1-ha770c72_1 
  bzip2              conda-forge/linux-64::bzip2-1.0.8-hda65f42_8 
  c-ares             conda-forge/linux-64::c-ares-1.34.6-hb03c661_0 
  ca-certificates    conda-forge/noarch::ca-certificates-2025.11.12-hbd8a1cb_0 
  cairo              conda-forge/linux-64::cairo-1.18.4-h3394656_0 
  curl               conda-forge/li



==> WARNING: A newer version of conda exists. <==
    current version: 25.9.1
    latest version: 25.11.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [ ]:
%%bash
# Activate conda vcf_filter environment
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh
conda activate vcf_filter

# Input VCF from 60% missingness threshold
input_vcf="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6_miss_60.recode.vcf"

# Middle 95% Depth Percentiles (calculated earlier)
min_depth=12.8302
max_depth=24.6604

# Output prefix
output_prefix="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6_miss_60_mid95percentile"

# Run VCFtools to filter by mean depth
vcftools --vcf "$input_vcf" \
         --min-meanDP "$min_depth" \
         --max-meanDP "$max_depth" \
         --recode \
         --out "$output_prefix"


VCFtools - 0.1.17
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6_miss_60.recode.vcf
	--max-meanDP 24.7
	--min-meanDP 12.8
	--out vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6_miss_60_mid95percentile
	--recode

After filtering, kept 53 out of 53 Individuals
Outputting VCF file...
After filtering, kept 94503 out of a possible 99691 Sites
Run Time = 36.00 seconds


In [59]:
%%bash
grep 'ZSB' vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05.imiss | awk '{print $1}' > vcf_file/zsb_samples.txt

In [62]:
%%bash
# Activate conda vcf_filter environment
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh
conda activate vcf_filter

# Define input & output
input_vcf="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6_miss_60_mid95percentile.recode.vcf"
remove_list="vcf_file/zsb_samples.txt"

output_prefix="vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6_miss_60_mid95percentile_noZSB"

# Run vcftools to remove ZSB individuals
vcftools --vcf "$input_vcf" \
         --remove "$remove_list" \
         --recode \
         --out "$output_prefix"

echo "Done: ZSB samples removed. Output = ${output_prefix}.recode.vcf"


Done: ZSB samples removed. Output = vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6_miss_60_mid95percentile_noZSB.recode.vcf



VCFtools - 0.1.17
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6_miss_60_mid95percentile.recode.vcf
	--remove vcf_file/zsb_samples.txt
	--out vcf_file/machali_Aligned_rangeWideMerge_strelka_update2_BENGAL_mac3_passOnly_biallelicOnly_rmvIndels_minMAF0Pt05_chr_E2_minDP3_minQ30_minGQ30_hwe_0.05_imiss_0.6_miss_60_mid95percentile_noZSB
	--recode

Excluding individuals in 'exclude' list
After filtering, kept 48 out of 53 Individuals
Outputting VCF file...
After filtering, kept 94503 out of a possible 94503 Sites
Run Time = 35.00 seconds
